In [2]:
from transformers import BertTokenizerFast, BertForPreTraining,TFBertForTokenClassification ,TFTrainer, TFTrainingArguments
import os
import tensorflow as tf
import pandas as pd
import datasets

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
from datasets import load_dataset
dataset = load_dataset("german_legal_entity_recognition","all",ignore_verifications =True , download_mode="force_redownload")

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.39M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset german_legal_entity_recognition downloaded and prepared to /root/.cache/huggingface/datasets/german_legal_entity_recognition/all/1.0.0/1d4ec33053d6b0f788de20aa9010bf6f5a16ea6682b83ddd79ab82773348658c. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset = dataset["train"].train_test_split(test_size=0.2)

In [6]:
labels = dataset["train"].features[f"ner_tags"].feature.names

In [7]:
#Load Model and Tokenizer
with tf.device('GPU'):
    tokenizer = BertTokenizerFast.from_pretrained('./Legal_TOK/')
    model = TFBertForTokenClassification.from_pretrained('./Legal_NER/', num_labels=len(labels) )

All model checkpoint layers were used when initializing TFBertForTokenClassification.

All the layers of TFBertForTokenClassification were initialized from the model checkpoint at ./Legal_NER/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [8]:
def tokenize_and_align_labels(examples):
    #print(examples.keys())
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [9]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/54 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [10]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [182]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer,max_length=512 ,padding='max_length', return_tensors="tf",label_pad_token_id =-100)

tf_train_set  = full_train_dataset.to_tf_dataset(
    columns=["attention_mask","token_type_ids", "input_ids", "labels"],
    shuffle=True,
    batch_size=6,
    collate_fn=data_collator,
)

tf_test_set  = full_eval_dataset.to_tf_dataset(
    columns=["attention_mask","token_type_ids", "input_ids", "labels"],
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator,
)


In [183]:
from keras import backend as K
import sys
true = 0
pred = 0
def balanced_recall(y_true, y_pred):
    global pred
    global true
    """This function calculates the balanced recall metric
    recall = TP / (TP + FN)
    """
    true = y_true
    pred = y_pred
     
     
    print((y_true.shape))
    recall_by_class = 0
    # iterate over each predicted class to get class-specific metric
 
    return 1


In [186]:
import tensorflow as tf
model.compile(optimizer="adam",metrics=[balanced_recall ]  )

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [187]:
model.fit(tf_train_set,batch_size=6)

(6, None)
(6,)
(6, None)
(6,)


KeyboardInterrupt: 

In [138]:

p = model.predict(i)

In [135]:
label = ""
obb = ""
for i in tf_test_set.take(1):
    obb = i
    label = i["labels"][0]

In [174]:
tf_test_set.shape

AttributeError: 'TakeDataset' object has no attribute 'shape'